In [1]:
# Import libraries

# install BeautifulSoup and Geocoder

!pip install BeautifulSoup4
!pip install geocoder
!pip install geopy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
#Web scraping information from Wikipedia.. 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
#soup=BeautifulSoup(page)
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')

In [6]:
#Create Table from the Wikepedia HTML Codes
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append((cells[0].find(text=True)).strip())
        B.append((cells[1].find(text=True)).strip())
        C.append((cells[2].find(text=True)).strip())


In [7]:
#Create Data Frame
df=pd.DataFrame([A,B,C])
df=df.transpose()
df.columns=(['PostalCode','Borogh','NeighbourHood'])
print(df.shape)

(180, 3)


In [8]:
df=df[df.Borogh!='Not assigned']
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)
print(df.shape)

(103, 3)


In [9]:
#check if the DataFrame postal code is duplicate. If yes, merge them into one row. but, there are no duplicates.
dupl=pd.DataFrame(df['PostalCode'].value_counts())
dupl=dupl[dupl.PostalCode!=1]
dupl

,PostalCode


In [11]:
#Downloaded geo.csv to get Lat and Longitude. Innerjoin 2 dataframes. df + df_geo into neighborhood_df with all information
df_geo=pd.read_csv("D:\geo.csv")
df_geo.rename(columns={'Postal Code':
                      'PostalCode'}, inplace=True)
neighborhood_df=pd.merge(df,df_geo,on='PostalCode',how='inner')

In [21]:
neighborhood_df.head()

,PostalCode,Borogh,NeighbourHood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
neighborhood_df.shape

(103, 5)

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhood_df['Borogh'].unique()), neighborhood_df.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


In [14]:
#Create a Map for Toronto  with neighborhoods.

address = 'Toronto, TOR'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7627912, -79.4064452.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borogh, neighborhood in zip(neighborhood_df['Latitude'], neighborhood_df['Longitude'], neighborhood_df['Borogh'], neighborhood_df['NeighbourHood']):
    label = '{}, {}'.format(neighborhood, borogh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
#Select neighborhoods containing "Toronto" - filtering only toronto to create a more cleaner map specific to toronto central areas

toronto_df=neighborhood_df[neighborhood_df['Borogh'].str.contains('Toronto')]

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borogh, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borogh'], toronto_df['NeighbourHood']):
    label = '{}, {}'.format(neighborhood, borogh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2